# Numerical Integration

A great deal of financial problems such as bond valuation (especially with changing interest rate) and valuation of plain vanilla European Option boils down to simple integration problems. However, not all integral problems are well defined with analytical solution. In this section, I explored three simple numerical approaches for approximating those function without analytical integral solution.

Three approximation (more advanced to be updated soon) methods are discussed as well as bond, Black-Sholes valuation examples.

* Midpoint method (constant approximation)
* Trapezoidal method (linear approximation)
* Simpson's method (Quadratic approximation)

In [25]:
import numpy as np
from math import exp, pi, log
import pandas as pd
% precision 8

'%.8f'

## Simple Integration

Integral values are commonly approximated by dividing integral interval into smaller sub-intervals. The three methods mentioned above differed in terms of how to approximate sub-inteval value.

$ \int_a^b \mathrm{e}^{-x^2}\,\mathrm{d}x $ is used to evaluate these methods.

In [26]:
def func(x):
   return exp(-x**2)

0.00000000

** Midpoint method **

A constant was used to approximate the integral in each sub-interval.

In [3]:
def midpoint(a,b,n):
    # a: left end point for the integral
    # b: right end point for the integral
    # n: number of partition intervals
    h = (b-a)/n   # sub-interval length
    I_midpoint = 0
    for i in range(n):
      I_midpoint = I_midpoint + func(a+h*(i+0.5))  
    return  h*I_midpoint  

** Trapezoidal method **

Linear interpolation was used to approximate the integral in each sub-interval.

In [4]:
def trapezoidal(a,b,n):
    # a: left end point for the integral
    # b: right end point for the integral
    # n: number of partition intervals
    h = (b-a)/n   # sub-interval length
    I_trap = func(a)/2 + func(b)/2
    for i in range(n-1):
        I_trap = I_trap + func(a+h*(i+1))
    return h*I_trap

** Simpson's method **

Quadratic interpolation was used to approximate the integral in each sub-interval.

In [5]:
def simpson(a,b,n):
    # a: left end point for the integral
    # b: right end point for the integral
    # n: number of partition intervals
    h = (b-a)/n   # sub-interval length
    I_simpson = func(a)/6 + func(b)/6
    for i in range(n-1):
        I_simpson = I_simpson + func(a+h*(i+1))/3
    for i in range(n):
        I_simpson = I_simpson + 2*func(a+h*(i+0.5))/3
    return h*I_simpson

** A first simple comparison **

In [6]:
Intervals = [4,8,16,32,64,128,256,512]
Midpoint = np.zeros(8)
Trapezoidal = np.zeros(8)
Simpson = np.zeros(8)
for i in range(8):
    Midpoint[i] = midpoint(0,2,Intervals[i])
for i in range(8):
    Trapezoidal[i] = trapezoidal(0,2,Intervals[i])
for i in range(8):
    Simpson[i] = simpson(0,2,Intervals[i])
Comparison = pd.DataFrame(Intervals)
Comparison['Midpoint Method'] = Midpoint
Comparison['Trapezoidal Method'] = Trapezoidal
Comparison['Simpson Method'] = Simpson
Comparison.columns = ['No. Intervals', 'Midpoint Method','Trapezoidal Method','Simpson Method']
Comparison

,No. Intervals,Midpoint Method,Trapezoidal Method,Simpson Method
0,4,0.882789,0.880619,0.882066
1,8,0.882269,0.881704,0.882080
2,16,0.882129,0.881986,0.882081
3,32,0.882093,0.882058,0.882081
4,64,0.882084,0.882075,0.882081
5,128,0.882082,0.882080,0.882081
6,256,0.882082,0.882081,0.882081
7,512,0.882081,0.882081,0.882081


** Convergence Speed **

1. If the function f(x) is 2nd order differentiable and is continous after differentiion. Midpoint and Trapezoidal methods converge in 1/n^2
2. If the function f(x) is 4th order differentiable and is continous after differentiion. Midpoint and Trapezoidal methods converge in 1/n^4

In [7]:
# an algorithm used to calculate the number of intervals to reach certain tolerance range.
# Suppose we have a tolerance rate of .0.0000005
%time
tolerance = 0.0000005
# Number of intervals taken for midpoint
n = 4
I_old = midpoint(0,2,n)
n= 2*n
I_new = midpoint(0,2,n)
while abs(I_new-I_old)>tolerance:
    I_old=I_new
    n=2*n
    I_new=midpoint(0,2,n)
print('Midpoint method converges into tolerance zoone after:', n)
print('integral value is: ', I_new)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
Midpoint method converges into tolerance zoone after: 512
integral value is:  0.8820814373412922


In [8]:
# Number of intervals taken for Trapezoidal
%time
n = 4
I_old = trapezoidal(0,2,n)
n= 2*n
I_new = trapezoidal(0,2,n)
while abs(I_new-I_old)>tolerance:
    I_old=I_new
    n=2*n
    I_new=trapezoidal(0,2,n)
print('Trapezoidal method converges into tolerance zoone after:', n)
print('integral value is: ', I_new)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
Trapezoidal method converges into tolerance zoone after: 512
integral value is:  0.8820812976045025


In [9]:
# Number of intervals taken for Simpson method
%time
n = 4
I_old = simpson(0,2,n)
n= 2*n
I_new = simpson(0,2,n)
while abs(I_new-I_old)>tolerance:
    I_old=I_new
    n=2*n
    I_new=simpson(0,2,n)
print('Simpson method converges into tolerance zoone after:', n)
print('integral value is: ', I_new)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
Simpson method converges into tolerance zoone after: 32
integral value is:  0.8820813868806548


It is clearly to see that, the Simpson method converges in a faster speed. This is because the function is defferentiable to the 4th power.

## Bond valuation example

Here, suppose we have a semi-annual coupon bond with 5% coupon rate and 4 years to maturity. The price was caculated from:
* spot rate curve
* instantaneous rate curve

Also, duration and convexity can be obtained

In [10]:
# Bond feature:
n = 8  # number of total payment 
T_pmt = np.linspace(.5,4,8)  # payment date
V_pmt = [5,5,5,5,5,5,5,105] # pmt made on each pmt date

** Valuation given spot rate curve **

Valuation given a known spot rate curve is a simple discount of each payment

In [11]:
spot_rate = [.03,0.2,.04,.06,.06,.04,.05,.04]
Bond_Price = 0
disc = np.zeros(8)
for i in range(8):
    disc[i] = exp(-T_pmt[i]*spot_rate[i])
    Bond_Price = Bond_Price + V_pmt[i]*disc[i]
Bond_Price

120.57316333

** Valuation given instantaneous rate **

Can be calculated with integration

In [12]:
inst_rate = [.03,0.2,.04,.06,.06,.04,.05,.04]
Bond_Price = 0
disc = np.zeros(8)
num = np.zeros(8)
def func(x):
    return .05
for i in range(8):
    n = 4
    I_old = simpson(0,T_pmt[i],n)
    n = 2*n
    I_new = simpson(0,T_pmt[i],n)
    while abs(I_old-I_new)>.0000005:    # set tolerance as 0.0000005
        I_old = I_new
        n=2*n
        I_new = simpson(0,T_pmt,n)
    num[i] = I_new
    disc[i] = exp(-I_new)
    Bond_Price = Bond_Price + V_pmt[i]*disc[i]

Bond_Price

117.67563978

** Duration and Convexity **

In [13]:
T = 4 # bond maturity (in years)
n = 8 # number of cash flows
T_pmt = np.linspace(.5,4,8)  # payment date
V_pmt = [5,5,5,5,5,5,5,105] # pmt made on each pmt date
y = .05 # yield to maturity

B=0 # bond price
D=0 # duration of the bond
C=0 # convexity of the bond

disc = np.zeros(n)
for i in range(n):
    disc[i] = exp(-y*T_pmt[i])
    B=B+V_pmt[i]*disc[i]
    D=D+T_pmt[i]*V_pmt[i]*disc[i]
    C=C+V_pmt[i]*disc[i]*T_pmt[i]**2
D=D/B
C=C/B
print('Bond price is: ', B)
print('Duration is: ', D)
print('Convexity is: ', C)

Bond price is:  117.675639777
Duration is:  3.44761344052
Convexity is:  12.982325355


## Black-Schole Example

** Cumulative Prob Function **

First we need to define the function used for standard normal cumulative probability function. One thing should be mentioned is that scipy provides integral functions. Hoever I used Abramowitz and Stegun to approximate this function. This method has a approximation error smaller than 0.00000075. 

In [18]:
def standard_normal_cum(t):
    z = abs(t)
    y=1/(1+0.2316419*z)
    a1=0.319381530
    a2=-0.356563782
    a3=1.781477937
    a4=-1.821255978
    a5=1.330274429
    m=1-exp(-0.5*t**2)*(a1*y+a2*y**2+a3*y**3+a4*y**4+a5*y**5)/(2*pi)**0.5
    if t>0:
        ans = m
    else:
        ans=1-m
    return ans

In [22]:
# test the accuracy
print(standard_normal_cum(0))
print(standard_normal_cum(2.33))
print(standard_normal_cum(1.96))

0.49999999947519136
0.9900969469515918
0.9750021748433644


** Black-Scholes **

The starting point of derivative pricing

In [33]:
def Black_Scholes(t, s, k, T, sig, r, q, option):
    # t: present time
    # s: current spot price
    # k: strike price
    # T: maturity time
    # sig: volatility
    # r: risk free rate
    # q: continous dividend rate of underlying asset 
    # option type 'C' for call and 'P' for put
    d1 = (log(s/k)+(r-q+0.5*sig**2))/(sig*(T-t)**0.5)
    d2 = d1 - (sig*(T-t)**0.5)
    if option == 'C':
        Option = s*exp(q*(t-T))*standard_normal_cum(d1)-k*exp(r*(t-T))*standard_normal_cum(d2)
    if option == 'P':
        Option = k*exp(r*(t-T))*standard_normal_cum(-d2)-s*exp(q*(t-T))*standard_normal_cum(-d1)
    return Option

In [34]:
Black_Scholes(0,42,40,0.5,0.3,0.03,0.05,'P')

2.66569295

In [35]:
Black_Scholes(0,42,40,0.5,0.3,0.03,0.05,'C')

4.22423167